# SARIMA(X) Modeling With PCA

Next Step:
Update model perfomance chart

In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import itertools
import warnings
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from IPython.display import display, HTML

## Data Preparation

In [42]:
# Load data
df = pd.read_excel('data/PCA Mastersheet.xlsx')

# Ensure date is datetime and set index
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)
df.index = pd.date_range(start=df.index[0], periods=len(df), freq='MS')
df.columns = df.columns.str.strip()

In [43]:
macro_list = ['PC1_macro', 'PC2_macro']
asset_list= ['PC1_crypto', 'PC2_crypto', 'VIX', 'MOVE']
train_end = '2024-01-01'

In [44]:
ar_orders = {}
ar_orders['PC1_macro'] = {'p': 1, 'd': 1, 'q': 0, 'P': 1, 'D': 1, 'Q': 0, 
                          'PC1_crypto_lag': 1, 'PC2_crypto_lag': 1, 'VIX_lag': 7, 'MOVE_lag': 1}
ar_orders['PC2_macro'] = {'p': 1, 'd': 1, 'q': 1, 'P': 1, 'D': 1, 'Q': 0, 
                          'PC1_crypto_lag': 0, 'PC2_crypto_lag': 0, 'VIX_lag': 0, 'MOVE_lag': 0}

# ar_orders['VIX'] = {'p': 1, 'd': 0,'q': 0, 'P': 1, 'D': 1, 'Q': 0}
# ar_orders['MOVE'] = {'p': 1, 'd': 1,'q': 0, 'P': 1, 'D': 0, 'Q': 0}


## Running SARIMA(X) Model

In [48]:
def run_model(df, macro, asset, plot):
    order_dict = ar_orders.get(macro, {'p': 1, 'd': 1, 'q': 0, 'P': 1, 'D': 1, 'Q': 0})
    
    # Unpack ARIMA and seasonal orders
    p = order_dict['p']
    d = order_dict['d']
    q = order_dict['q']
    P = order_dict['P']
    D = order_dict['D']
    Q = order_dict['Q']

    # Extract asset-specific lag
    asset_lag_key = f"{asset}_lag"
    asset_lag = order_dict.get(asset_lag_key, 0)

    ### ==== AR Data: Use only macro series ==== ###
    df_macro = df[[macro]].dropna().copy()
    target_ar = df_macro[macro]
    train_endog_ar = target_ar[:train_end]
    test_endog_ar = target_ar[train_end:]

    ### ==== ARX Data: Use macro + asset ==== ###
    df_temp = df[[macro, asset]].dropna().copy()

    # Create lagged asset columns
    for lag in range(1, asset_lag + 1):
        df_temp[f'{asset}_lag{lag}'] = df_temp[asset].shift(lag)

    exog_cols = [asset] + [f'{asset}_lag{lag}' for lag in range(1, asset_lag + 1)]
    df_temp = df_temp.dropna()

    exog = df_temp[exog_cols]
    target_arx = df_temp[macro]

    train_endog_arx = target_arx[:train_end]
    train_exog = exog[:train_end]
    test_endog_arx = target_arx[train_end:]
    test_exog = exog[train_end:]

    ### ==== Fit AR and ARX Models ==== ###
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always", ConvergenceWarning)

        ar_model = SARIMAX(train_endog_ar, order=(p, d, q), seasonal_order=(P, D, Q, 12))
        ar_result = ar_model.fit(disp=False)

        arx_model = SARIMAX(train_endog_arx, exog=train_exog, order=(p, d, q), seasonal_order=(P, D, Q, 12))
        arx_result = arx_model.fit(disp=False)

        for warning in w:
            if issubclass(warning.category, ConvergenceWarning):
                print(f"[WARNING] Convergence issue in macro: {macro}, asset: {asset}")

    ### ==== Forecasts ==== ###
    pred_ar = ar_result.get_forecast(steps=len(test_endog_ar)).predicted_mean
    conf_int_ar = ar_result.get_forecast(steps=len(test_endog_ar)).conf_int()

    pred_arx = arx_result.get_forecast(steps=len(test_endog_arx), exog=test_exog).predicted_mean
    conf_int_arx = arx_result.get_forecast(steps=len(test_endog_arx), exog=test_exog).conf_int()

    # Align index for plotting
    pred_ar.index = test_endog_ar.index
    pred_arx.index = test_endog_arx.index
    conf_int_ar.index = test_endog_ar.index
    conf_int_arx.index = test_endog_arx.index

    ### ==== Plotting ==== ###
    if plot:
        plt.figure(figsize=(10, 5))
        plt.plot(target_ar, label='Actual ' + macro, color='black')
        plt.plot(pred_ar, label=f'Forecasted {macro} (AR only)', linestyle='--', color='blue')
        plt.fill_between(pred_ar.index, conf_int_ar.iloc[:, 0], conf_int_ar.iloc[:, 1], color='blue', alpha=0.1)
        plt.plot(pred_arx, label=f'Forecasted {macro} (ARX with {asset})', linestyle='--', color='red')
        plt.fill_between(pred_arx.index, conf_int_arx.iloc[:, 0], conf_int_arx.iloc[:, 1], color='red', alpha=0.1)
        plt.title("Out-of-Sample Forecast")
        plt.legend()
        plt.tight_layout()
        plt.show()

        plt.figure(figsize=(10, 5))
        plt.plot(test_endog_ar, label='Actual ' + macro, marker='o', color='black')
        plt.plot(pred_ar, label=f'AR Forecast', linestyle='--', marker='x', color='blue')
        plt.plot(pred_arx, label=f'ARX Forecast', linestyle='--', marker='s', color='red')
        plt.title("Forecast vs Actual (Test Period)")
        plt.xlabel("Date")
        plt.ylabel(macro)
        plt.legend()
        plt.tight_layout()
        plt.show()

    ### ==== Metrics ==== ###
    metrics = [
        {
            'Model': 'AR',
            'MAE': mean_absolute_error(test_endog_ar, pred_ar),
            'RMSE': np.sqrt(mean_squared_error(test_endog_ar, pred_ar)),
            'R2': r2_score(test_endog_ar, pred_ar),
            'MAPE (%)': mean_absolute_percentage_error(test_endog_ar, pred_ar) * 100,
            'Order': f'({p},{d},{q})'
        },
        {
            'Model': 'ARX',
            'MAE': mean_absolute_error(test_endog_arx, pred_arx),
            'RMSE': np.sqrt(mean_squared_error(test_endog_arx, pred_arx)),
            'R2': r2_score(test_endog_arx, pred_arx),
            'MAPE (%)': mean_absolute_percentage_error(test_endog_arx, pred_arx) * 100,
            'Order': f'({p},{d},{q})'
        }
    ]
    return pd.DataFrame(metrics).set_index('Model')


In [55]:
results_list = []
for macro in macro_list:
        for asset in asset_list:
                # Run model, plot=False to skip plotting in batch run
                metrics_df = run_model(df.copy(), macro, asset, plot=False)
                # metrics_df is a DataFrame with index Model (AR, ARX) and columns MAE, RMSE, R2, MAPE, Order
                # Add macro and asset columns for clarity
                metrics_df['Macro'] = macro
                metrics_df['Asset'] = asset
                
                results_list.append(metrics_df.reset_index())
# Combine all results into one DataFrame
final_results = pd.concat(results_list, ignore_index=True)

# Rearrange columns to show Model, Macro, Asset, and errors only
final_results = final_results[['Model', 'Macro', 'Asset', 'MAE', 'RMSE', 'R2', 'MAPE (%)']]

# Format float columns for better readability
float_cols = ['MAE', 'RMSE', 'R2']
final_results[float_cols] = final_results[float_cols]

[WARNING] Convergence issue in macro: PC1_macro, asset: VIX
[WARNING] Convergence issue in macro: PC1_macro, asset: MOVE
[WARNING] Convergence issue in macro: PC2_macro, asset: PC2_crypto


In [57]:
final_results

,Model,Macro,Asset,MAE,RMSE,R2,MAPE (%)
0,AR,PC1_macro,PC1_crypto,984.237581,1143.344565,-27.964347,4.656080
1,ARX,PC1_macro,PC1_crypto,821.351194,1059.414695,-23.868035,3.884003
2,AR,PC1_macro,PC2_crypto,984.237581,1143.344565,-27.964347,4.656080
3,ARX,PC1_macro,PC2_crypto,891.729437,1032.183098,-22.606031,4.221489
4,AR,PC1_macro,VIX,984.237581,1143.344565,-27.964347,4.656080
5,ARX,PC1_macro,VIX,609.609620,732.570700,-10.890731,2.901489
6,AR,PC1_macro,MOVE,984.237581,1143.344565,-27.964347,4.656080
7,ARX,PC1_macro,MOVE,549.208641,700.084170,-9.859504,2.599159
8,AR,PC2_macro,PC1_crypto,98.854498,106.911753,0.728595,0.481666
9,ARX,PC2_macro,PC1_crypto,84.582230,95.134843,0.785095,0.411680


In [58]:
# Reshape for comparison
df_wide = final_results.pivot_table(
    index=['Macro', 'Asset'],
    columns='Model',
    values=['MAE', 'RMSE', 'R2', 'MAPE (%)']
)

df_wide.columns = ['_'.join(col).strip() for col in df_wide.columns.values]
df_wide.reset_index(inplace=True)

# Define better = lower RMSE, lower MAE, higher R²
df_wide['ARX_better_RMSE'] = df_wide['RMSE_ARX'] < df_wide['RMSE_AR']
df_wide['ARX_better_MAE'] = df_wide['MAE_ARX'] < df_wide['MAE_AR']
df_wide['ARX_better_MAPE'] = df_wide['MAPE (%)_ARX'] < df_wide['MAPE (%)_AR']
df_wide['ARX_better_R2']  = df_wide['R2_ARX']  > df_wide['R2_AR']

# Filter: only combinations where ARX is better by **all** metrics
better_all = df_wide[
    (df_wide['ARX_better_RMSE']) &
    (df_wide['ARX_better_MAE']) &
    (df_wide['ARX_better_MAPE']) &
    (df_wide['ARX_better_R2'])
]

# Display results
if not better_all.empty:
    print("Combinations where ARX (with asset) outperforms AR on all metrics (MAE, RMSE, MAPE, R²):")
    display(better_all[['Macro', 'Asset', 'MAE_AR', 'MAE_ARX', 'RMSE_AR', 'RMSE_ARX', 'MAPE (%)_AR', 'MAPE (%)_ARX', 'R2_AR', 'R2_ARX']])
else:
    print("No combination found where ARX beats AR across MAE, RMSE, and R².")


# Partial wins
print("\n Combinations where ARX has lower MAE:")
display(df_wide[df_wide['ARX_better_MAE']][['Macro', 'Asset', 'MAE_AR', 'MAE_ARX']])

print("\n Combinations where ARX has lower RMSE:")
display(df_wide[df_wide['ARX_better_RMSE']][['Macro', 'Asset', 'RMSE_AR', 'RMSE_ARX']])

print("\n Combinations where ARX has lower MAPE:")
display(df_wide[df_wide['ARX_better_MAPE']][['Macro', 'Asset', 'MAPE (%)_AR', 'MAPE (%)_ARX']])

print("\n Combinations where ARX has higher R²:")
display(df_wide[df_wide['ARX_better_R2']][['Macro', 'Asset', 'R2_AR', 'R2_ARX']])

Combinations where ARX (with asset) outperforms AR on all metrics (MAE, RMSE, MAPE, R²):


,Macro,Asset,MAE_AR,MAE_ARX,RMSE_AR,RMSE_ARX,MAPE (%)_AR,MAPE (%)_ARX,R2_AR,R2_ARX
0,PC1_macro,MOVE,984.237581,549.208641,1143.344565,700.084170,4.656080,2.599159,-27.964347,-9.859504
1,PC1_macro,PC1_crypto,984.237581,821.351194,1143.344565,1059.414695,4.656080,3.884003,-27.964347,-23.868035
2,PC1_macro,PC2_crypto,984.237581,891.729437,1143.344565,1032.183098,4.656080,4.221489,-27.964347,-22.606031
3,PC1_macro,VIX,984.237581,609.609620,1143.344565,732.570700,4.656080,2.901489,-27.964347,-10.890731
5,PC2_macro,PC1_crypto,98.854498,84.582230,106.911753,95.134843,0.481666,0.411680,0.728595,0.785095
6,PC2_macro,PC2_crypto,98.854498,77.731121,106.911753,87.677150,0.481666,0.378423,0.728595,0.817468
7,PC2_macro,VIX,98.854498,90.066895,106.911753,102.500698,0.481666,0.438364,0.728595,0.750529



 Combinations where ARX has lower MAE:


,Macro,Asset,MAE_AR,MAE_ARX
0,PC1_macro,MOVE,984.237581,549.208641
1,PC1_macro,PC1_crypto,984.237581,821.351194
2,PC1_macro,PC2_crypto,984.237581,891.729437
3,PC1_macro,VIX,984.237581,609.609620
5,PC2_macro,PC1_crypto,98.854498,84.582230
6,PC2_macro,PC2_crypto,98.854498,77.731121
7,PC2_macro,VIX,98.854498,90.066895



 Combinations where ARX has lower RMSE:


,Macro,Asset,RMSE_AR,RMSE_ARX
0,PC1_macro,MOVE,1143.344565,700.084170
1,PC1_macro,PC1_crypto,1143.344565,1059.414695
2,PC1_macro,PC2_crypto,1143.344565,1032.183098
3,PC1_macro,VIX,1143.344565,732.570700
5,PC2_macro,PC1_crypto,106.911753,95.134843
6,PC2_macro,PC2_crypto,106.911753,87.677150
7,PC2_macro,VIX,106.911753,102.500698



 Combinations where ARX has lower MAPE:


,Macro,Asset,MAPE (%)_AR,MAPE (%)_ARX
0,PC1_macro,MOVE,4.656080,2.599159
1,PC1_macro,PC1_crypto,4.656080,3.884003
2,PC1_macro,PC2_crypto,4.656080,4.221489
3,PC1_macro,VIX,4.656080,2.901489
5,PC2_macro,PC1_crypto,0.481666,0.411680
6,PC2_macro,PC2_crypto,0.481666,0.378423
7,PC2_macro,VIX,0.481666,0.438364



 Combinations where ARX has higher R²:


,Macro,Asset,R2_AR,R2_ARX
0,PC1_macro,MOVE,-27.964347,-9.859504
1,PC1_macro,PC1_crypto,-27.964347,-23.868035
2,PC1_macro,PC2_crypto,-27.964347,-22.606031
3,PC1_macro,VIX,-27.964347,-10.890731
5,PC2_macro,PC1_crypto,0.728595,0.785095
6,PC2_macro,PC2_crypto,0.728595,0.817468
7,PC2_macro,VIX,0.728595,0.750529


## Testing Individual Combination

In [61]:
macro = 'PC1_macro'
asset = 'VIX'
run_model(df.copy(), macro, asset, plot=False)

[WARNING] Convergence issue in macro: PC1_macro, asset: VIX


,MAE,RMSE,R2,MAPE (%),Order
Model,,,,,
AR,984.237581,1143.344565,-27.964347,4.656080,"(1,1,0)"
ARX,609.609620,732.570700,-10.890731,2.901489,"(1,1,0)"


## Finding Optimal Asset Lag

In [53]:
# def find_optimal_lag(df, macro, asset, max_lag=10, verbose=False):
#     best_lag = None
#     best_improvement = np.inf
#     best_metrics = None

#     results = []

#     for lag in range(0, max_lag + 1):
#         # Temporarily override lag

#         try:
#             metrics = run_model(df, macro, asset, lag, plot = False)
#             ar = metrics.loc['AR']
#             arx = metrics.loc['ARX']

#             delta_mape = arx['MAPE (%)'] - ar['MAPE (%)']
#             results.append({
#                 'Lag': lag,
#                 'ΔMAPE': delta_mape,
#             })

#             if delta_mape < best_improvement:
#                 best_lag = lag
#                 best_improvement = delta_mape
#                 best_metrics = metrics

#             if verbose:
#                 print(f"Lag {lag}: ΔMAPE = {delta_mape:.2f}")

#         except Exception as e:
#             print(f"Lag {lag}: Failed with error: {e}")
#             continue

#     results_df = pd.DataFrame(results)
#     return best_lag, best_improvement, results_df, best_metrics


In [54]:
# for macro in ar_orders.keys():
#     best_lag, _, _, _ = find_optimal_lag(df, macro, asset="MOVE", max_lag=10)
    
#     if best_lag is not None:
#         ar_orders[macro]["MOVE_lag"] = best_lag
#     else:
#         ar_orders[macro]["MOVE_lag"] = np.nan 